<a href="https://colab.research.google.com/github/akshayjain777/AI/blob/master/ResNet_practise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F ## activation function
import torchvision ##contains dataset
import pdb

class baseBlock(torch.nn.Module):
  expansion=1
  def __init__(self,input_planes,planes,stride=1,dim_change=None):
    super(baseBlock, self).__init__()
    #declare convolution layers with batch norms
    self.conv1 = torch.nn.Conv2d(input_planes,planes,stride=stride,kernel_size=3,padding=1)
    self.bn1= torch.nn.BatchNorm2d(planes)
    self.conv2 = torch.nn.Conv2d(planes,planes,stride=1,kernel_size=3,padding=1)
    self.bn2=torch.nn.BatchNorm2d(planes)
    self.dim_change=dim_change
  def forward(self,x):
    #save the residue
    res = x
    output=F.relu(self.bn1(self.conv1(x)))
    output=self.bn2(self.conv2(output))

    if self.dim_change is not None:
      res=self.dim_change(res)

    output += res
    output= F.relu(output)

    return output

class bottleNeck(torch.nn.Module):
  expansion=4
  def __init__(self,input_planes,planes,stride=1,dim_change=None):
    super(bottleNeck,self).__init__()
    self.conv1 = torch.nn.Conv2d(input_planes,planes,kernel_size=1,stride=1)
    self.bn1=torch.nn.BatchNorm2d(planes)
    self.conv2 = torch.nn.Conv2d(planes,planes,kernel_size=3,stride=stride,padding=1)
    self.bn2= torch.nn.BatchNorm2d(planes)
    self.conv3= torch.nn.Conv2d(planes,planes*self.expansion,kernel_size=1)
    self.dim_change=dim_change
  
  def forward(self,x):
    res=x
    output=F.relu(self.bn1(self.conv1(x)))
    output=F.relu(self.bn2(self.conv2(output)))
    output= self.bn3(self.conv3(output))

    if self.dim_change is not None:
      res=self.dim_change(res)

    output += res
    output= F.relu(output)

    return output

class ResNet(torch.nn.Module):
  def __init__(self,block,num_layers,classes=10):
    super(ResNet,self).__init__()
    #according to research paper
    self.input_planes=64
    self.conv1= torch.nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1)
    self.bn1= torch.nn.BatchNorm2d(64)
    self.layer1 = self._layer(block,64,num_layers[0],stride=1)
    self.layer2 =  self._layer(block,128,num_layers[1],stride=2)
    self.layer3 = self._layer(block,256,num_layers[2],stride=2)
    self.layers4 = self._layer(block,512,num_layers[3],stride=2)
    self.averagePool = torch.nn.AvgPool2d(kernel_size=4,stride=1)
    self.fc=torch.nn.Linear(512*block.expansion,classes)

  def _layer(self,block,planes,num_layers,stride=1):
    dim_change=None
    if stride!=1 or planes !=self.input_planes*block.expansion:
      dim_change = torch.nn.Sequential(torch.nn.Conv2d(self.input_planes,planes*block.expansion,kernel_xize=1,stride=stride),torch.nn.BatchNorm2d(planes*block.expansion))
    
    netLayers= []
    netlayers.append(block(self.input_planes,planes,stride=stride,dim_change=dim_change))
    self.input_palnes= planes * block.expansion
    for i in range(1,num_layers):
      netLayers.append(block(self.input_planes,planes))
      self.input_planes= planes * block.expansion
    
    return torch.nn.Sequential(*netLayers)
  
  def forward(self,x):
    x=

  
